# 준비

In [ ]:
import warnings
warnings.simplefilter('ignore')

# 라이브러리 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 데이터 읽어 들임
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
# 훈련 데이터의 확인
train.head()

In [ ]:
# 테스트 데이터의 확인
test.head()

In [ ]:
# 데이터 사이즈의 확인
print("train.shape{}, test.shape{}".format(train.shape, test.shape))

# 분석

## 통계량의 확인

In [ ]:
# 훈련 데이터의 통계량을 산출
train.describe(include='all')

In [ ]:
# テストデータの統計量を算出
test.describe(include='all')

## 정답 라벨과의 관계를 확인

In [ ]:
## 변수 사이의 상관 계수의 히트맵
fig = plt.figure(figsize=(8,7))
ax = sns.heatmap(train.corr(), annot=True, cmap='BrBG', vmin=-1, vmax=1)
# fig.savefig('titanic_EDA_corr_heatmap.png', bbox_inches='tight')

In [ ]:
## 카테고리 변수(Sex)와 Survived 의 관계
fig = plt.figure(figsize=(6,4))
ax = sns.barplot(y='Survived', x='Sex', data=train)
# fig.savefig('titanic_EDA_Sex_Survived.png', bbox_inches='tight')

In [ ]:
## 카테고리 변수(Embarked)와 Survived 의 관계
fig = plt.figure(figsize=(6,4))
sns.barplot(y='Survived', x='Embarked', data=train)
# fig.savefig('titanic_EDA_Embarked_Survived.png', bbox_inches='tight')

## 데이터의 결손 확인

In [ ]:
# 데이터의 결손 확인
print("●train\n{}".format(train.isnull().sum()))
print("●test\n{}".format(test.isnull().sum()))

In [ ]:
## 연령의 결손 및 Survived의 관계
train["Age_null"] = train["Age"].isnull()

fig = plt.figure(figsize=(6,4))
sns.barplot(y='Survived', x='Age_null', data=train)
# fig.savefig('titanic_EDA_AgeNull_Survived.png', bbox_inches='tight')

del train["Age_null"]

# 데이터 가공

In [ ]:
# ライブラリインポート
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

## 데이터 전처리

In [ ]:
# Name을 Title로 변환하는 함수
def replace_name(series):
    series = series.apply(lambda x: x.split(',')[1].split('.')[0].strip())
    series = series.replace(['Capt','Col','Major','Dr','Rev'], 'Officer')
    series = series.replace(['Don','Sir','the Countess','Lady','Dona'], 'Royalty')
    series = series.replace(['Mme','Ms'], 'Mrs')
    series = series.replace(['Mlle'], 'Miss')
    series = series.replace(['Jonkheer'], 'Master')
    return series

In [ ]:
# 카테고리 변수의 정의
categorical_features = ["Sex", "Embarked", "Title"]

In [ ]:
# 특징량 엔지니어링 실행
train = train.dropna(subset=['Embarked'])
df = pd.concat([train, test])
df['Family'] = df['SibSp'] + df['Parch'] + 1
df["Title"] = replace_name(df["Name"])
df["Age_null"] = df["Age"].isnull()
df["Age"] = df["Age"].fillna(df["Age"].median())
df["Fare"] = df["Fare"].fillna(df[df["Pclass"]==3]["Fare"].median())
df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Parch'])
oe = OrdinalEncoder()
df[categorical_features] = oe.fit_transform(df[categorical_features]).astype(int)
train = df[df["Survived"].notnull()]
test = df[df["Survived"].isnull()].drop(columns=["Survived"])

## 데이터 세트 작성

In [ ]:
# 훈련 데이터를 학습 데이터와 검증 데이터로 분할
train, valid = train_test_split(train, test_size=0.2, stratify=train["Survived"], random_state=100)

In [ ]:
# 데이터의 보존
train.to_csv("train_proc.csv",index=None)
valid.to_csv("valid_proc.csv",index=None)
test.to_csv("test_proc.csv",index=None)

In [ ]:
# 설명 변수와 목적 변수로 분할
def make_Xy(df, col_y="Survived"):
    return df.drop(columns=[col_y]), df[col_y]

train_X, train_y = make_Xy(train)
valid_X, valid_y = make_Xy(valid)
test_X = test

# 모델의 학습

In [ ]:
# 라이브러리 임포트
from lightgbm import LGBMClassifier
import pickle as pkl

In [ ]:
# 모델 정의
model = LGBMClassifier(max_depth=4, colsample_bytree=0.5, 
                       reg_lambda=0.5, reg_alpha=0.5, 
                       importance_type="gain", random_state=100)

In [ ]:
# 학습의 실행
model.fit(
    train_X, train_y,
    eval_set=[(valid_X, valid_y)],
    early_stopping_rounds=50,
    verbose=10,
    categorical_feature=categorical_features
)

In [ ]:
# 정답률의 계산
print("Accuracy(train) {:.3f}".format(model.score(train_X, train_y)))
print("Accuracy(valid) {:.3f}".format(model.score(valid_X, valid_y)))

In [ ]:
# 모델의 보존
with open('lgbm_model.pkl', 'wb') as f:
    pkl.dump(model, f)

In [ ]:
# 중요도
fig = plt.figure(figsize=(6,4))
sns.barplot(x=model.feature_importances_, y=model.feature_name_)
# fig.savefig('titanic_feature_importance.png', bbox_inches='tight')

# LIME을 사용한 설명

In [ ]:
# 라이브러리 임포트
from lime.lime_tabular import LimeTabularExplainer

## 준비

In [ ]:
# LIME용 예측 계수의 준비
def predict_fn(X):
    if len(X.shape)==1:
        return model.predict_proba(X.reshape(1,-1))[0]
    else:
        return model.predict_proba(X)

In [ ]:
# 변수와 카테고리 대응 준비
class_names = ["Not Survived", "Survived"]
categorical_feature_idx = np.where(train_X.columns.isin(categorical_features))[0]
categorical_names = dict(zip(categorical_feature_idx , [list(lst) for lst in oe.categories_]))

## 실행

In [ ]:
# 설명용 클래스 정의
explainer = LimeTabularExplainer(training_data=train_X.values,
                                 feature_names=train_X.columns,
                                 categorical_features=categorical_feature_idx,
                                 categorical_names=categorical_names,
                                 class_names=class_names
                                )

In [ ]:
# 예측대상 데이터의 인덱스
i = 0

# 국소설명의 계산
exp = explainer.explain_instance(test_X.values[i], predict_fn, num_features=5)

In [ ]:
# jupyter notebook로 가시화
exp.show_in_notebook()

In [ ]:
# matplotlib형식으로 취득
plt = exp.as_pyplot_figure()
# plt.savefig('titanic_lime_matplotlib.png', bbox_inches='tight')

In [ ]:
# 수치를 취득
## LIME기능만 이용
exp_list = exp.as_list()
exp_list

In [ ]:
# 원래의 특징량 순서로 정렬
def get_exp_list_all(exp, explainer=explainer):
    exp_list_all = np.zeros(len(explainer.feature_names))
    for col_id, val in exp.local_exp[1]:
        exp_list_all[col_id] = val
    exp_list_all = list(zip(model.feature_name_, exp_list_all))
    return exp_list_all

exp_list_all = get_exp_list_all(exp)
exp_list_all

In [ ]:
i = 20
exp = explainer.explain_instance(test_X.values[i], predict_fn, num_features=5)
exp.show_in_notebook()

In [ ]:
i = 100
exp = explainer.explain_instance(test_X.values[i], predict_fn, num_features=5)
exp.show_in_notebook()

# 분석１：남성이지만 Survive로 예측된 승객의 조사

In [ ]:
# 분석용 테이블을 작성
pred = predict_fn(test_X)
result = pd.DataFrame({"pred":pred[:,0], "Sex":test["Sex"]})

In [ ]:
# 남성이지만 Survive로 예측된 사례를 취득
target_idx = result[result["Sex"]==1].sort_values("pred").index

In [ ]:
# LIME 실행
i = target_idx[0]
exp = explainer.explain_instance(test_X.values[i], predict_fn, num_features=5)
exp.show_in_notebook()

# 분석２：kernel width조정

In [ ]:
# 설명용 클래스 정의
## kernel_width=0.1
explainer = LimeTabularExplainer(training_data=train_X.values,
                                 feature_names=train_X.columns,
                                 categorical_features=categorical_feature_idx,
                                 categorical_names=categorical_names,
                                 class_names=class_names, 
                                 kernel_width=0.1
                                )

In [ ]:
i = target_idx[0]
exp = explainer.explain_instance(test_X.values[i], predict_fn, num_features=5)
exp.show_in_notebook(show_all=False)

In [ ]:
# 설명용 클래스 정의
## kernel_width=0.5
explainer = LimeTabularExplainer(training_data=train_X.values,
                                 feature_names=train_X.columns,
                                 categorical_features=categorical_feature_idx,
                                 categorical_names=categorical_names,
                                 class_names=class_names, 
                                 kernel_width=0.5
                                )

In [ ]:
i = target_idx[0]
exp = explainer.explain_instance(test_X.values[i], predict_fn, num_features=5)
exp.show_in_notebook(show_all=False)

In [ ]:
# 설명용 클래스 정의
## kernel_width=5.0
explainer = LimeTabularExplainer(training_data=train_X.values,
                                 feature_names=train_X.columns,
                                 categorical_features=categorical_feature_idx,
                                 categorical_names=categorical_names,
                                 class_names=class_names, 
                                 kernel_width=5.0
                                )

In [ ]:
i = target_idx[0]
exp = explainer.explain_instance(test_X.values[i], predict_fn, num_features=5)
exp.show_in_notebook(show_all=False)